In [1]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()


In [29]:
from pathlib import Path
from haystack.components.converters import TextFileToDocument

converter = TextFileToDocument()

docs = converter.run(sources=[Path("a.txt")])


In [30]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()


In [33]:
[docs]

[{'documents': [Document(id=9fead048b460d2ce47b2dd0af8a3316fd92acfc464cb39ba2cf670fb6a85c9a3, content: '淫荡少妇白洁 第八章　一路风流荡少妇
   　　和张敏吃过晚饭从那富丽堂皇的大酒店回到家中，白洁心里有些不舒服，却不知道自己哪里不快乐，只是不想说话，路上夫妻两人都没有说话，白洁自顾扭着微微翘起的小屁股，...', meta: {'file_path': 'a.txt'})]}]

In [34]:
docs_with_embeddings = doc_embedder.run({"documents":[docs]})
document_store.write_documents(docs_with_embeddings["documents"])


TypeError: SentenceTransformersDocumentEmbedder expects a list of Documents as input.In case you want to embed a list of strings, please use the SentenceTransformersTextEmbedder.

: 

In [7]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


In [9]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

from haystack.utils import Secret



generator = OpenAIGenerator(model="gpt-4o-mini",api_key=Secret.from_env_var('TABBY_API_KEY'),api_base_url="https://hk.imtass.me/v1",)


In [10]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)


In [11]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [12]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}},include_outputs_from='retriever')

print(response["llm"]["replies"][0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

While we don't have a definitive visual depiction, we can piece together some information about what the Colossus of Rhodes likely looked like. 

* **Head and Face:**  The head probably featured curly hair with evenly spaced bronze or silver flames radiating out, similar to images on contemporary Rhodian coins because this was a common depiction of Helios at the time.
* **Pose:** The statue was likely standing, perhaps with one hand shielding its eyes like a person looking at the sun. 

* **Materials:** It was constructed with iron tie bars and brass plates for the outer shell, giving it a metallic appearance.


Unfortunately, no complete artistic renderings of the Colossus of Rhodes survive, leaving us with educated guesses based on descriptions and artistic trends of the time period. 



In [ ]:
from haystack import Pipeline
from haystack.components.converters import MarkdownToDocument
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.utils import Secret

from milvus_haystack import MilvusDocumentStore
from milvus_haystack.milvus_embedding_retriever import MilvusEmbeddingRetriever


document_store = MilvusDocumentStore(
    connection_args={"uri": "./milvus.db"},
    # connection_args={"uri": "http://localhost:19530"},
    # connection_args={"uri": YOUR_ZILLIZ_CLOUD_URI, "token": Secret.from_env_var("ZILLIZ_CLOUD_API_KEY")},
    drop_old=True,
)
